In [7]:
# Google ColabやLinux環境の場合
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' | tee /etc/apt/sources.list.d/google-chrome.list
!apt-get update
!apt-get install google-chrome-stable
!pip install selenium webdriver-manager

OK
deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main
Hit:1 http://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... 

In [13]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
import datetime
import re

# ==========================================
# Step 2. HTMLから株価データを抽出する関数（強化版）
# ==========================================
def extract_stock_data(html):
    """
    HTMLを解析してデータを抽出する関数
    クラス名が変わっても「始値」などのラベル文字を探してデータを取得します
    """
    soup = BeautifulSoup(html, 'html.parser')

    # 抽出するデータの入れ物
    extracted_data = {}
    keywords = ['始値', '高値', '安値', '終値']

    # 【戦略1】ページ内の全テキストから日付を探す (YYYY/MM/DD形式)
    text_content = soup.get_text(separator=' ', strip=True)
    date_match = re.search(r'(\d{4}/\d{1,2}/\d{1,2})', text_content)

    if not date_match:
        return None

    date_str = date_match.group(1)

    # 【戦略2】「始値」などのキーワードを探し、その近くの数値を取得する
    # <td>始値</td><td>38,000</td> のような構造や、<div>始値 38,000</div> などを想定
    for key in keywords:
        # キーワードを含む要素をすべて探す
        elements = soup.find_all(string=re.compile(key))
        for el in elements:
            # その要素の親や、隣の要素から数値を探す
            parent = el.parent
            if not parent: continue

            # パターンA: 同じ要素内に数値がある場合 (例: "始値: 38,000")
            text = parent.get_text(strip=True)
            # 数値(カンマ含む)を抽出する正規表現
            num_match = re.search(r'[\d,]+\.\d+|[\d,]+', text.replace(key, ''))

            # パターンB: 隣の要素に数値がある場合 (例: <td>始値</td><td>38,000</td>)
            if not num_match:
                # 兄弟要素（隣のタグ）をいくつか確認
                for sibling in parent.find_next_siblings(limit=2):
                    sib_text = sibling.get_text(strip=True)
                    num_match = re.search(r'[\d,]+\.\d+|[\d,]+', sib_text)
                    if num_match:
                        break

            if num_match:
                extracted_data[key] = num_match.group(0)
                break # 見つかったら次のキーワードへ

    # 4つのデータが揃っているか確認
    if all(k in extracted_data for k in keywords):
        try:
            # 日付フォーマット変換 (2024/01/01 -> 2024-01-01)
            dt = datetime.datetime.strptime(date_str, '%Y/%m/%d')
            formatted_date = dt.strftime('%Y-%m-%d')

            return [
                formatted_date,
                extracted_data['始値'],
                extracted_data['高値'],
                extracted_data['安値'],
                extracted_data['終値']
            ]
        except Exception:
            return None

    return None

# ==========================================
# Step 3. Webページへアクセスしデータを抽出する関数
# ==========================================
def get_stock_values(driver, url):
    driver.get(url)
    time.sleep(5) # ページ読み込み待ち

    try:
        # チャート要素を特定
        chart_element = driver.find_element(By.CSS_SELECTOR, '.highcharts-container')
    except:
        print("エラー: チャート要素が見つかりません。")
        return []

    # チャートを画面中央にスクロール（超重要）
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", chart_element)
    time.sleep(2)

    # チャートのサイズ情報を取得
    chart_size = chart_element.size
    width = chart_size['width']
    height = chart_size['height']

    actions = ActionChains(driver)

    # グラフの「右側」かつ「線がありそうな高さ」に移動
    # width * 0.9 (右端付近), height * 0.4 (上から40%付近)
    start_x = width * 0.9
    start_y = height * 0.4

    # マウスを移動
    actions.move_to_element_with_offset(chart_element, start_x, start_y)
    actions.perform()
    time.sleep(1) # ホバー反応待ち

    stock_data_list = []
    seen_dates = set()

    # データを取得するループ
    loop_count = 30
    print(f"データ取得中 ({loop_count}回)...")

    for i in range(loop_count):
        # 現在のHTMLを取得して解析
        html = driver.page_source
        data = extract_stock_data(html)

        if data:
            date_str = data[0]
            if date_str not in seen_dates:
                stock_data_list.append(data)
                seen_dates.add(date_str)
                # print(f"  -> 取得: {data[0]} {data[4]}") # デバッグ用

        # マウスを左へ少しずらす (ピクセル単位)
        # ※ 反応が悪い場合は -10 など大きくしてみてください
        actions.move_by_offset(-5, 0)
        actions.perform()

        # 描画更新待ち（早すぎるとデータが変わらない）
        time.sleep(0.5)

    return stock_data_list

# ==========================================
# Step 4. mainコード
# ==========================================
if __name__ == "__main__":
    start_time = time.time()

    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    # User-Agentを偽装（Bot判定回避のため重要）
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # タイムアウト設定
    driver.implicitly_wait(10)

    target_url = "https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month"

    try:
        result_data = get_stock_values(driver, target_url)

        end_time = time.time()
        elapsed_time = end_time - start_time

        print("-" * 40)
        print(f"スクレイピングにかかった時間: {elapsed_time:.2f}秒")
        print("-" * 40)
        print("日付, 始値, 高値, 安値, 終値")

        if not result_data:
            print("データを取得できませんでした。")
            print("対策: コード内の actions.move_by_offset の値を調整するか、start_y の高さを変えてみてください。")
        else:
            # 取得できたデータを表示
            for item in result_data:
                print(item)

    finally:
        driver.quit()

データ取得中 (30回)...
----------------------------------------
スクレイピングにかかった時間: 45.46秒
----------------------------------------
日付, 始値, 高値, 安値, 終値
['2026-02-02', '53,575.37', '54,247.15', '52,655.18', '52,655.18']
